In [1]:
import tqdm.notebook as tqdm
import numpy as np
import scipy
import sklearn

# Spark

In [2]:
! /home/jovyan/start-hadoop.sh

jovyan
 * Starting OpenBSD Secure Shell server sshd
start-stop-daemon: unable to set gid to 0 (Operation not permitted)
   ...fail!
 * sshd is running
Starting namenodes on [localhost]
localhost: Warning: Permanently added 'localhost' (ED25519) to the list of known hosts.
localhost: namenode is running as process 157.  Stop it first and ensure /tmp/hadoop-jovyan-namenode.pid file is empty before retry.
Starting datanodes
localhost: Warning: Permanently added 'localhost' (ED25519) to the list of known hosts.
localhost: datanode is running as process 266.  Stop it first and ensure /tmp/hadoop-jovyan-datanode.pid file is empty before retry.
Starting secondary namenodes [69e835ffee64]
69e835ffee64: Warning: Permanently added '69e835ffee64' (ED25519) to the list of known hosts.
69e835ffee64: secondarynamenode is running as process 417.  Stop it first and ensure /tmp/hadoop-jovyan-secondarynamenode.pid file is empty before retry.
Starting resourcemanager
resourcemanager is running as process

In [3]:
# connect, context, session

import findspark
findspark.init()

import pyspark
sc = pyspark.SparkContext(appName='jupyter')

from pyspark.sql import SparkSession, Row
se = SparkSession(sc)


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2023-04-04 08:23:06,183 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


## HDFS

In [4]:
! hdfs dfs -df -h

Filesystem               Size   Used  Available  Use%
hdfs://localhost:9000  58.4 G  4.8 G     30.5 G    8%


In [5]:
! hdfs dfs -ls /

Found 2 items
drwxrwx---   - root   supergroup          0 2023-04-03 17:45 /tmp
drwxr-xr-x   - jovyan supergroup          0 2023-04-03 17:45 /user


## Outbrain click prediction dataseet

https://www.kaggle.com/c/outbrain-click-prediction/data - you need to register here

In [ ]:
# https://github.com/Kaggle/kaggle-api
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
# instruction where to get ~/.kaggle/kaggle.json file


  Using cached kaggle-1.5.12-py3-none-any.whl
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12


In [12]:
! mkdir -p ~/.kaggle

In [19]:
%%writefile ~/.kaggle/kaggle.json
{"username":"riyaromanova","key":"4e8b52865473938cfb28d73be8bfcc3d"}


Overwriting /home/jovyan/.kaggle/kaggle.json


In [20]:
! chmod 600 ~/.kaggle/kaggle.json

In [21]:
! pip install -U urllib3 kaggle==1.5.3
! kaggle competitions download -c outbrain-click-prediction -f page_views_sample.csv.zip
! kaggle competitions download -c outbrain-click-prediction -f events.csv.zip 
! kaggle competitions download -c outbrain-click-prediction -f documents_topics.csv.zip 
! kaggle competitions download -c outbrain-click-prediction -f documents_entities.csv.zip 
! kaggle competitions download -c outbrain-click-prediction -f documents_meta.csv.zip 
! kaggle competitions download -c outbrain-click-prediction -f clicks_test.csv.zip
! kaggle competitions download -c outbrain-click-prediction -f clicks_train.csv.zip
! kaggle competitions download -c outbrain-click-prediction -f documents_categories.csv.zip
! kaggle competitions download -c outbrain-click-prediction -f promoted_content.csv.zip

  Using cached urllib3-1.26.15-py2.py3-none-any.whl (140 kB)
100%|███████████████████████████████████████▊| 148M/149M [00:15<00:00, 10.7MB/s]
100%|████████████████████████████████████████| 149M/149M [00:15<00:00, 10.1MB/s]
100%|████████████████████████████████████████| 478M/478M [00:48<00:00, 11.1MB/s]
100%|████████████████████████████████████████| 478M/478M [00:48<00:00, 10.2MB/s]
100%|████████████████████████████████████████| 121M/121M [00:12<00:00, 9.93MB/s]
100%|████████████████████████████████████████| 121M/121M [00:12<00:00, 9.87MB/s]
100%|████████████████████████████████████████| 126M/126M [00:13<00:00, 10.9MB/s]
100%|████████████████████████████████████████| 126M/126M [00:13<00:00, 10.1MB/s]
100%|██████████████████████████████████████| 15.5M/15.5M [00:02<00:00, 10.0MB/s]
100%|██████████████████████████████████████| 15.5M/15.5M [00:02<00:00, 7.15MB/s]
100%|███████████████████████████████████████▊| 135M/135M [00:14<00:00, 10.3MB/s]
100%|████████████████████████████████████████| 1

In [22]:
! unzip '*.zip'
! rm -rf *.zip

Archive:  documents_entities.csv.zip
  inflating: documents_entities.csv  

Archive:  promoted_content.csv.zip
  inflating: promoted_content.csv    

Archive:  documents_topics.csv.zip
  inflating: documents_topics.csv    

Archive:  documents_categories.csv.zip
  inflating: documents_categories.csv  

Archive:  clicks_train.csv.zip
  inflating: clicks_train.csv        

Archive:  clicks_test.csv.zip
  inflating: clicks_test.csv         

Archive:  page_views_sample.csv.zip
  inflating: page_views_sample.csv   

Archive:  events.csv.zip
  inflating: events.csv              

Archive:  documents_meta.csv.zip
  inflating: documents_meta.csv      

9 archives were successfully processed.


In [23]:
! hdfs dfs -put page_views_sample.csv
! hdfs dfs -put events.csv
! hdfs dfs -put documents_topics.csv
! hdfs dfs -put documents_entities.csv
! hdfs dfs -put documents_meta.csv
! hdfs dfs -put clicks_test.csv
! hdfs dfs -put clicks_train.csv
! hdfs dfs -put documents_categories.csv
! hdfs dfs -put promoted_content.csv

## Evaluation Assignment

Data: outbrain click prediction

Tasks:
Using Spark RDD, DataFrame API and Python, calculate:

**1**. Top 10 most visited document_ids in the page_views_sample log

**2**. How many users have at least 2 different traffic_sources in the page_views_sample log (note the value is not a count, it's an encoded enum)

**3***. Top 10 most visited topic_ids in page_views_sample log (use documents_topics table)

The submission format is the result.json json file with top_10_documents, users and top_10_topics keys.
For TOP-10 results, the answer must be written in the form of a sheet ordered from TOP-1 to TOP-10 with an id.

result.json example:

    {
        "top_10_documents": [
            111,
            222,
            333,
            ...,
            1010
        ],
        "users": 10000,
        "top_10_topics": [
            11,
            22,
            33,
            ...,
            101
        ]
    }

In [6]:
df = se.read.csv("page_views_sample.csv", header=True)

In [7]:
#top 10 docs
views_log = df.rdd
(
    views_log
    .map(lambda x: (x[1],1))
    .reduceByKey(lambda a, b: a + b)
    .takeOrdered(10, lambda x: -x[1])
)

[('1811567', 429551),
 ('234', 179692),
 ('42744', 156231),
 ('1858440', 112140),
 ('1780813', 109624),
 ('60164', 104941),
 ('1790442', 91420),
 ('1877626', 80309),
 ('1821895', 79956),
 ('732651', 74630)]

In [8]:
df.show(5)

+--------------+-----------+---------+--------+------------+--------------+
|          uuid|document_id|timestamp|platform|geo_location|traffic_source|
+--------------+-----------+---------+--------+------------+--------------+
|1fd5f051fba643|        120| 31905835|       1|          RS|             2|
|8557aa9004be3b|        120| 32053104|       1|       VN>44|             2|
|c351b277a358f0|        120| 54013023|       1|       KR>12|             1|
|8205775c5387f9|        120| 44196592|       1|       IN>16|             2|
|9cb0ccd8458371|        120| 65817371|       1|   US>CA>807|             2|
+--------------+-----------+---------+--------+------------+--------------+
only showing top 5 rows



In [22]:
#users with at least 2 trafic sources
(
    views_log
    .map(lambda x: (x[0], list(x[5])))
    .sortByKey(ascending=False)
    .reduceByKey(lambda a, b: a + b)
    .map(lambda x: (x[0], set(x[1])))
    .filter(lambda x: len(x[1]) > 1)
    .count()
)

98080

In [27]:
documents_topics = se.read.csv("documents_topics.csv", header=True)

In [28]:
documents_topics.show(5)

+-----------+--------+------------------+
|document_id|topic_id|  confidence_level|
+-----------+--------+------------------+
|    1595802|     140|0.0731131601068925|
|    1595802|      16|0.0594164867373976|
|    1595802|     143|0.0454207537554526|
|    1595802|     170|0.0388674285182961|
|    1524246|     113| 0.196450402209685|
+-----------+--------+------------------+
only showing top 5 rows



In [29]:
from pyspark.sql import SparkSession, Row
se = SparkSession(sc)

In [30]:
documents_topics.registerTempTable('docs')
df.registerTempTable('logs')

In [34]:
topic_ids = se.sql("""
    SELECT topic_id, 1
    from logs l join docs d on d.document_id = l.document_id

""")
topic_ids.show()

+--------+---+
|topic_id|  1|
+--------+---+
|     183|  1|
|     184|  1|
|     235|  1|
|      45|  1|
|     166|  1|
|     111|  1|
|     183|  1|
|     184|  1|
|     235|  1|
|      45|  1|
|     166|  1|
|     111|  1|
|     183|  1|
|     184|  1|
|     235|  1|
|      45|  1|
|     166|  1|
|     111|  1|
|     183|  1|
|     184|  1|
+--------+---+
only showing top 20 rows



In [37]:
#top 10 topics
(
    topic_ids.rdd
    .reduceByKey(lambda a, b: a + b)
    .takeOrdered(10, lambda x: -x[1])
)

[('20', 1429253),
 ('16', 1425838),
 ('216', 1160563),
 ('136', 1099382),
 ('140', 971983),
 ('143', 919136),
 ('36', 855793),
 ('97', 839328),
 ('8', 819599),
 ('269', 727145)]

In [24]:
data = {
    "top_10_documents": [
        1811567,
        234,
        42744,
        1858440,
        1780813,
        60164,
        1790442,
        1877626,
        1821895,
        732651
    ],
    "users": 98080,
    "top_10_topics": [
        20,
        16,
        216,
        136,
        140,
        1423
}

In [25]:

import json
with open('result.json', 'w') as f:
    f.write(json.dumps(data))

In [26]:
!curl -F file=@result.json "51.250.54.133:80/MDS-LSML1/riyarmnv/w4/1"

0.67
 Wrong answer for top_10_topics
